### Pomocné třídní dekorátory
---

<br>

#### Metody v OOP

---

Jako ucelený seznam metod, které můžeme použít v rámci OOP se podívej na tyto metody:
1. **Instanční** metoda,
2. **třídní** metoda,
3. **statická** metoda.

<br>

Celou situaci lze vysvětlit na teoretické ukázce:

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def load_data(self):  # klasická metoda instance
        return "Calling instance method..", self
    
    @classmethod
    def parsing_files(cls):  # třídní metoda
        return "Calling class method..", cls
    
    @staticmethod
    def is_there_file():     # statická metoda
        return "Calling static method.."

In [ ]:
reader = KindleNotesParser()

<br>

##### Instační metoda
Metoda, která má mj. mezi parametry klíčový výraz `self`. Tím dovede zpřístupnit jak třídy (a její atributy), tak instance.

In [ ]:
print(
    reader,              # adresa objektu instance 'reader'
    reader.load_data,    # popis metody patřící instanci 'reader'
    reader.load_data(),  # spuštění, opět adresa
    sep="\n"
)

<br>

Takže pomocí předchozí ukázky můžeme říct, že **instanční metoda** je schopna zpřístupnit *původní třídu* a případně pracovat s jejími *atributy*.

<br>

##### Třídní metoda
Podobný zápis jako pro instační metodu, ale liší se dvěma zásadním rozdíly:
1. Dekorátor `@classmethod`,
2. parametr `cls`.

In [ ]:
print(
    reader.parsing_files,    # adresa objektu instance nyní chybí
    reader.parsing_files(),  # metodu spouští, třídy zpřístupní, ale instanci nezná
    sep="\n"
)

<br>

Jakmile použiješ **třídní metodu**, vidíš že máš přístup k objektu *původní třídy*, ale tentokrát není k dispozici odkaz (adresa) *instance*.

```
('Reading .txt file..', <__main__.KindleParser object at 0x7f0ed30782e0>)
('Spoustim metodu tridy', <class '__main__.KindleParser'>)
```

<br>

##### Statická metoda

1. Dekorátor `@staticmethod`,
2. chybí parametr `self`,
3. chybí parametr `cls`.

In [ ]:
print(
    reader.is_there_file,
    reader.is_there_file(),
    sep="\n"
)

In [ ]:
print(KindleNotesParser.load_data())

In [ ]:
print(KindleNotesParser.parsing_files())

In [ ]:
print(KindleNotesParser.is_there_file())

<br>

Pokud spustíš a prozkoumáš **statickou metodu**, můžeš si ověřit, že tato metoda nemá přístup ani k *původní třídě*, ani k její *instanci*.

<br>

##### Shrnutí k metodám

---
1. **instanční metoda** - může upravit nejenom objekty instance, ale i třídy (na začátku vidí jak třídu, tak instanci),
2. **třídní metoda (@classmethod)** - může upravit objekty třídy, ale nemůže upravovat objekty instancí (vidí třídu, ale ne instanci),
3. **statická metoda (@staticmethod)** - nemůže upravovat ani objekty instancí, ani objekty třídy (nevidí ani třídu, ani instanci).

<br>

##### Praktické ukázky

---

###### Instanční metoda

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    notes: list = []  # upravení třídního atributu
    
    def __init__(self, file: str, data: str):
        self.file = file
        self.data = data
#         self.notes = list()  # upravení instan. atributu
        
    def load_data(self):
        return self.notes.append(self.data)

In [ ]:
first_note = KindleNotesParser("poznamky.txt", "Moje první poznámka k ...")
second_note = KindleNotesParser("poznamky_nove.txt", "Druhá poznámka ke knížce ...")

first_note.load_data()
second_note.load_data()

print(first_note.notes, second_note.notes, sep="\n")

<br>

Je jedno, který atribut budeš chtít upravit. Díky **instanční metodě** můžeš pracovat jak s třídními, tak s instančními objekty.
<br>

###### Třídní metoda

In [ ]:
class KindleNotesParser:
    """Parse the data in the .txt file"""
    readed_files: int = 0
    
    def __init__(self, file: str):
        self.file = file
    
    @classmethod
    def parsing_files(cls, name):
        instance = cls(name)
        cls.readed_files += 1  
        print(f"Parsing data from: {name}")

In [ ]:
print(f"{KindleNotesParser.readed_files=}")

In [ ]:
KindleNotesParser.parsing_files("poznamky.txt")
KindleNotesParser.parsing_files("nove_poznamky.txt")
KindleNotesParser.parsing_files("poznpozn.txt")

In [ ]:
print(f"{KindleNotesParser.readed_files=}")

<br>

Pokud budeš chtít použít **třídní metody**, potom dávej pozor na to, že můžeš spravovat pouze třídní atributy.
<br>

###### Statická metoda

In [ ]:
import os

class KindleNotesParser:
    """Parse the data in the .txt file"""
    
    def __init__(self, file: str):
        self.file = file
    
    @staticmethod
    def is_there_file(name):
        print("The file exists!") if os.path.exists(name) else print("Does not exist!")


In [ ]:
parser = KindleNotesParser("")
parser.is_there_file("lesson01.ipynb")

In [ ]:
parser.is_there_file("lesson11.ipynb")

<br>

**Statická metoda** nepotřebuje vědět nic ani o třídě, ani o instanci. Pracuje s parametrem jako klasická funkce. Ale svým účelem spadá jako nástroj ke konkrétní třídě.

<br>

#### Vlastnosti třídy

---

Představ si situaci, kdy máš napsat převodník jednotek objemu. Z **litrů** na **pinty**(UK):

In [ ]:
class LiterConvertor:
    def __init__(self, liter: int = 0):
        self.liter = liter
        
    def to_pints(self):
        return self.liter * 1.759754

In [ ]:
bottle_volume = LiterConvertor()
bottle_volume.liter = 0.75
print(bottle_volume.liter)
print(bottle_volume.to_pints())

<br>

Kdykoliv se pokusíš tímto způsobem přepsat hodnotu, interpret pracuje stejně jako když přepisuješ hodnotu ve slovníku:

In [ ]:
 print(bottle_volume.__dict__)

In [ ]:
print(
    bottle_volume.liter,              # instační atribut
    bottle_volume.__dict__["liter"],  # slovníkový výběr podle klíče
    sep="\n"
)

<br>

Lepší implementace (pythonovější) v rámci objektově-orientovaného programování by vypadala následovně:

In [ ]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.set_volume_in_liter(liter)
        
    def to_pints(self):
        return self.get_volume_in_liter() * 1.759754
    
    def get_volume_in_liter(self):
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value

In [ ]:
volume = LiterConvertor(0.75)
print(
    volume.get_volume_in_liter(),
    volume.to_pints(),
    sep="\n"
)

In [ ]:
volume.set_volume_in_liter(-1)

<br>

Toto řešení nám dovolilo přidat omezení, kdy nechceme počítat objem s negativní hodnotou.

<br>

Bohužel se také objevil problém s ohledem na implementaci našeho vylepšení. Všechny ohlášení `obj.liter` je nutné přepsat na `obj.get_volume_in_liter()` a `obj.liter = val` na `obj.set_volume(val)`.

<br>

Taková úprava řešení může znamenat problém na desítky, stovky řádků.

<br>

Naštěstí můžeme s vlastnostmi třídy nakládat lépe!

In [ ]:
class LiterConvertor:
    def __init__(self, liter: float = 0.0):
        self.liter = liter
        
    def to_pints(self):
        return self.liter * 1.759754
    
    def get_volume_in_liter(self):
        print("Getting..")
        return self._liter
    
    def set_volume_in_liter(self, value: float):
        print("Setting..")
        if value < 0:
            raise ValueError("Cannot process negative number")
        self._liter = value
        
    liter = property(get_volume_in_liter, set_volume_in_liter)

In [ ]:
volume = LiterConvertor(0.75)
print(volume.liter)
print(volume.to_pints())

In [ ]:
volume.liter = -1

<br>

Pokaždé co tentokrát použijeme proměnnou `liter` dojde automaticky k zavolání metody `get_volume_in_liter()`.

<br>

Stejně tak, pokud budeš chtít přepast hodnotu v proměnnou `liter`, tak dojde ke spuštění metody `set_volume_in_liter()`.

<br>

Takže pokud použiješ funkci `property()`, nemusíš se omezovat na potřebné úpravy tvého stávajícího skriptu.

<br>


#### Dekorátory setter, getter, deleter

---

<br>

<br>

#### Podtržítka v Pythonu

---
Podtržítko je v syntaktický znak, který má nejeden význam pro Python.

<br>

##### Samotné podtržítko (skip)

In [ ]:
import time

def check_logging_messages(limit: int) -> None:
    """Check the current logging messages."""
    
    for sec in range(limit):
        print("Checking logging messages..")
        time.sleep(1)
        
check_logging_messages(5)

<br>

Proměnná `sec`, kterou jsme vytvořili v rámci funkce `check_logging_messages` nemá žádné využití.

<br>

Proto v Pythonu existuje symbol `_` pro označení nevyužívané proměnné:
```python
    for _ in range(limit):
        print("Checking logging messages..")
```

<br>

Případně je možné přeskakovat i u vícenásobného přiřazování:
```python
_, domena = "matous@gmail.com".split("@")
print(domena)
```

<br>

##### Slabé privátní objekty (*weak private*)

Některé jazyky implicitně dovolují práci s pomocí **privátních proměnných** (~Java).

<br>

Python tuto funkcionalitu **nepodporuje**:

In [ ]:
import time

class ProtocolChecker:
    def __init__(self, limit: int):
        self._limit = limit
        
    def check_protocol(self) -> None:
        for _ in range(self._limit):
            print("Checking routing protocol...")
            time.sleep(1)

In [ ]:
checker = ProtocolChecker(5)
print(checker._limit)

checker._limit = 10
print(checker._limit)

<br>

Je možné metodu nebo proměnnou označit pomocí jednoho podtržítka, ale to pouze naznačuje ostatní programátorům, že tento objekt je interní a není *DOPORUČENÉ* jej přepisovat.

<br>

##### Silné privátní proměnné (*strong private*)

Pomocí dvou podtržítek uživatel může definovat **chráněné proměnné** a tím předejít přepsání nebo přetypování.

<br>


In [1]:
import time

class ProtocolChecker:
    def __init__(self, limit: int):
        self.__limit = limit
        
    def check_protocol(self) -> None:
        for _ in range(self.__limit):
            print("Checking routing protocol...")
            time.sleep(1)

In [2]:
checker = ProtocolChecker(5)
print(f"{checker.__limit=}")

AttributeError: 'ProtocolChecker' object has no attribute '__limit'

In [3]:
print(checker.__dict__)

{'_ProtocolChecker__limit': 5}


In [5]:
checker._ProtocolChecker__limit = 10
print(checker.__dict__)

{'_ProtocolChecker__limit': 10}


<br>

Nicméně ani toto řešení není 100%. Pomocí metody `__dict__` je možné zjistit, jaké proměnné (..a hodnoty) má instance k dispozici a dohledáme přejmenovanou proměnnou.

<br>

##### Vlastní jméno (*any*)

Pokud se ti bude krýt klíčové slovo se jménem proměnné, interpret ti bude vracet syntaktickou výjimku:

In [10]:
class Employer:
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

class = Employer("Matous", "matous@gmail.com")

SyntaxError: invalid syntax (<ipython-input-10-9e5025b3f93d>, line 6)

<br>

Pokud chceš tomuto nežádoucímu scénaři zabránit, můžeš použít podtržítko jako příponu za klíčovým výrazem:
```python
class_ = Employer("Matous", "matous@gmail.com")
```

<br>

##### Magické metody (double-underscore methods ~ dunder methods)

Magické metody jsou speciální metody v Pythonu, na kterých stojí veškerá práce s objekty.

---